In [1]:

import pytorch_lightning as pl
from omegaconf import OmegaConf

from nemo.collections.asr.models import EncDecCTCModel
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"]="0"

[NeMo W 2021-02-04 15:18:30 experimental:27] Module <class 'nemo.collections.asr.losses.ctc.CTCLoss'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno -3] Temporary failure in name resolution>
[nltk_data] Error loading cmudict: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[NeMo W 2021-02-04 15:18:31 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-02-04 15:18:31 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-02-04 15:18:31 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimen

In [23]:
cfg = OmegaConf.load('conf/config-5x5_all.yaml') # 学習するときに読ませたconfigファイルからconfigを得る

In [24]:
trainer = pl.Trainer(**cfg['trainer'])
asr_model = EncDecCTCModel(cfg=cfg['model'], trainer=trainer) # 重みランダムなモデルを生成

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [8]


[NeMo I 2021-02-04 12:20:12 collections:173] Dataset loaded with 651505 files totalling 397.23 hours
[NeMo I 2021-02-04 12:20:12 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-02-04 12:20:14 collections:173] Dataset loaded with 36693 files totalling 22.18 hours
[NeMo I 2021-02-04 12:20:14 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-02-04 12:20:14 features:235] PADDING: 16
[NeMo I 2021-02-04 12:20:14 features:251] STFT using torch


In [25]:
sd = torch.load('nemo_experiments/QuartzNet5x5_CSJ/2021-02-04_01-44-40/checkpoints/QuartzNet5x5_CSJ-last.ckpt') # 重みをロード
asr_model.load_state_dict(sd['state_dict']) 

<All keys matched successfully>

In [26]:
files = ['/opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I05_HS.wav', '/opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I10_HS.wav', '/opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I15_HS.wav', '/opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I20_HS.wav', '/opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I25_HS.wav', '/opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I30_HS.wav', '/opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I35_HS.wav', '/opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I40_HS.wav', '/opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I45_HS.wav', '/opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I50_HS.wav']
for fname, transcription in zip(files, asr_model.transcribe(paths2audio_files=files)): # 推論
    print(f"Audio in {fname} was recognized as: {transcription}")

Audio in /opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I05_HS.wav was recognized as: 天然記念物休の規模という学者の発言もあった
Audio in /opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I10_HS.wav was recognized as: 病床の母親が誰かに手紙の大質を頼む
Audio in /opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I15_HS.wav was recognized as:  昼前に合宿所の電話が鳴った
Audio in /opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I20_HS.wav was recognized as:  うこれは漁民達の日常経験である
Audio in /opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I25_HS.wav was recognized as:  つその水源の呼称は炒め付けられ悲鳴を上げる元気もない
Audio in /opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I30_HS.wav was recognized as: 最後までどうも不思議な程おいしいなと思い続けた
Audio in /opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I35_HS.wav was recognized as: 最も編集会議にはその方が都合が良い
Audio in /opt/storage/datasets/audio/japanese/JNAS/WAVES_HS/F001/PB/BF001I40_HS.wav was recognized as:  みんなはそれから彼が入院中人付き合いが悪くどうも苦手だったという票

In [7]:
files = ['/src/CSJ/WAV/core/A01M0099.wav']
for fname, transcription in zip(files, asr_model.transcribe(paths2audio_files=files)):
    print(f"Audio in {fname} was recognized as: {transcription}")

Audio in /src/CSJ/WAV/core/A01M0099.wav was recognized as:  つ すーすすー えーとまず研究の背景ですが えーこれまでの話者照合の研究では えー他の話者や録音音声による詐称につきましては えー考慮されてきましたが えー合成音声による詐称につきましては殆ど考慮されてきませんでしたしかし えー近年 えー合成技術の発達により え高頻質で えー多様な話者性を持つ音声合成音声を あ音声合成することができるようになってきたことから えー合成音声による詐称の句エ討も必要ととなってきました んでー我々はこれまでに えーテキスト指て型の話者照合システムに対して えーエ延メモに基づく音声合成システムを用いた え詐称の検討を行なってまいりました えーその結果 えー話者照合システムの登録話者による数文章の音声をこの合成システムの学習データーとして用いた場合でも えー合成音声の受例率率が非常に高くなるという結果を示しました えーしかしこれまでの え実験では話者照合システムで えースペクトル情報しか用いていなかったこともあり え合成音声の励振源として白色雑音を用いていましたそこで えー ふ本研究では話者照合システムで えーピッチ情報も利用した場合について検討を行ないましたで えーとまず えー複合性音声による えー しょ詐称の けん詐称の要素を示しま えーとまず話者照合システムでは えーに えー照合の要求がありましたら適当なテキストを生成し えー話者に提示しますね予め学習してあるその話者の音素英延メムをこの えー接続してこのテキストに対応する文一目を生成し えー入力音声から特徴パラメーターを抽出して尤度計算し えー閾値と比較することによってえ えー受理または えー棄却の判定を行ないますで えー合成音声で詐称する場合には えーまずこの えー合成の基本単位となる音素エジ延メムを登録話者の音声数文章で学習しておきますそして えー照合システムからー えーテキストが提示されましたらこの音素エジ延メムを えーこのテキストに従って接続しパラメーターを えースペクトルパラメーターよびピッチを生成して えー合成フィルターにより音声を合成しますでこの合成音声を えー話者照合システムへの入力としまえこれまでの検討では えー え話者照合システムで えー特徴パラメ